# Elasticsearch Queries


## Setup


In [1]:
from datetime import datetime, timezone
from elasticsearch import Elasticsearch
import sys
sys.path.append('..')
import os
os.chdir('../')
from connectors.elk import Elk
elk = Elk()
es:Elasticsearch = elk.es

Elasticsearch: Connection sucessful


## Users not connected from more than X days


In [ ]:
from datetime import datetime, timedelta

# Definición de la consulta con top_hits para obtener detalles adicionales
query = {
    "size": 0,
    "aggs": {
        "users": {
            "terms": {
                "field": "userId.keyword",
                "size": 10000  # Ajusta este número según la cantidad esperada de usuarios
            },
            "aggs": {
                "latest_login": {
                    "max": {
                        "field": "@timestamp"
                    }
                },
                "top_user_info": {
                    "top_hits": {
                        "sort": [
                            {
                                "@timestamp": {
                                    "order": "desc"
                                }
                            }
                        ],
                        "_source": {
                            "includes": ["userDisplayName", "deviceDetail.displayName"]
                        },
                        "size": 1
                    }
                }
            }
        }
    }
}

# Realiza la consulta
# Asegúrate de cambiarlo por el nombre real de tu índice
new_index_name = "logs-ms_singins"
result = es.search(index=new_index_name, body=query)

# Extrae y ordena los resultados por la fecha de última conexión de manera ascendente
users_last_login = []
for bucket in result['aggregations']['users']['buckets']:
    user_id = bucket['key']
    last_login_timestamp = bucket['latest_login']['value_as_string']
    user_info = bucket['top_user_info']['hits']['hits'][0]['_source']
    user_display_name = user_info.get('userDisplayName', 'N/A')
    device_display_name = user_info.get(
        'deviceDetail', {}).get('displayName', 'N/A')
    users_last_login.append(
        (user_id, user_display_name, device_display_name, last_login_timestamp))

# Ordena por la fecha de última conexión de manera ascendente
users_last_login_sorted = sorted(users_last_login, key=lambda x: x[3])

# Define el formato de fecha utilizado en tus timestamps
fecha_formato = "%Y-%m-%dT%H:%M:%S.%fZ"

# Calcula la fecha límite para los usuarios inactivos (hace más de 30 días)
limite_inactividad = datetime.now() - timedelta(days=30)

# Filtra los usuarios inactivos por más de 30 días
usuarios_inactivos = []
for user in users_last_login_sorted:
    user_id, user_display_name, device_display_name, last_login_str = user
    last_login = datetime.strptime(last_login_str, fecha_formato)

    if last_login < limite_inactividad:
        usuarios_inactivos.append(user)

# Imprime los usuarios inactivos
for user in usuarios_inactivos:
    print(user)

## User searcher


In [ ]:
response = es.search(
    index="ms_users",
    query={
        "multi_match": {
            "query":    "victor gutierrez",
            "fields": ["displayName", "deviceDisplayName"]
        }
    }
)

print(json.dumps(dict(response), indent=2))
print(json.dumps(response["hits"]["hits"][0]["_source"]["mail"], indent=2))

## Last connection from a user


In [ ]:
# Define el userId para el cual quieres obtener la última conexión
user_id = "21bf1acf-0ebe-420e-b385-9d4014a11406"

# Construye la consulta
query = {
    "query": {
        "match": {
            "userId": user_id
        }
    },
    "sort": [
        {
            "@timestamp": {
                "order": "desc"
            }
        }
    ],
    "size": 1
}

# Realiza la consulta al índice deseado
new_index_name = "logs-ms_singins"
response = es.search(index=new_index_name, body=query)


print(json.dumps(dict(response), indent=4))

# Extrae y muestra la información relevante de la respuesta
if response["hits"]["hits"]:
    last_login = response["hits"]["hits"][0]["_source"]
    print(f"Última conexión de {last_login['userDisplayName']} ({last_login['userPrincipalName']}):")
    print(f"Fecha y hora: {last_login['@timestamp']}")
    print(f"IP: {last_login['ipAddress']}")
    print(f"Dispositivo: {last_login['deviceDetail']['displayName']}, {last_login['deviceDetail']['operatingSystem']}")
else:
    print("No se encontraron conexiones para el usuario especificado.")

# Reindex

In [ ]:
new_index_name = "ms_users"
old_index_name = "ms_users2"

reindex_body = {
    "source": {"index": old_index_name},
    "dest": {"index": new_index_name}
}
es.reindex(body=reindex_body)

print(f"Datos reindexados de {old_index_name} a {new_index_name}.")

## With a filter

In [3]:
new_index_name = "logs-ms_signins_interactive"
old_index_name = "logs-ms_singins"

field_key = "isInteractive"
field_value = True

reindex_body = {
  "source": {
    "index": old_index_name,
    "query": {
      "match": {
        field_key: field_value
      }
    }
  },
  "dest": {
    "index": new_index_name
  }
}

es.reindex(body=reindex_body, wait_for_completion=False)

print(f"Los datos se estan reindexando de {old_index_name} a {new_index_name}.")

Los datos se estan reindexando de logs-ms_singins a logs-ms_signins_interactive.
